In [1]:
from pycalphad import Database, Model
from pycalphad.core.utils import make_callable
import pycalphad.variables as v
import numba
import numpy as np

In [2]:
dbf = Database('Al-Ni/Al-Ni-Dupin-2001.tdb')
mod = Model(dbf, ['AL', 'NI', 'VA'], 'FCC_L12')

In [3]:
from sympy import lambdify
from pycalphad.core.utils import NumPyPrinter
@numba.vectorize
def where(condition, x, y):
    if condition:
        return x
    else:
        return y
func = lambdify(tuple(mod.variables), mod.ast, dummify=True,
                          modules=[{'where': where}, 'numpy'], printer=NumPyPrinter)
pure_func = lambdify(tuple(mod.variables), mod.ast, dummify=True,
                          modules=['numpy'], printer=NumPyPrinter)

In [4]:
vec_func = numba.vectorize(['float64({})'.format(','.join(['float64'] * len(tuple(mod.variables))))], nopython=True, target='parallel')(func)


x1 = np.random.rand(5, 1, 1)*2000
x2 = np.random.rand(5, 5, 3000)
x3 = np.random.rand(5, 5, 3000)
x4 = np.random.rand(5, 5, 3000)
x5 = np.random.rand(5, 5, 3000)
x6 = np.random.rand(5, 5, 3000)
#np.testing.assert_allclose(vec_func(x1, x2, x3, x4, x5, x6), pure_func(x1, x2, x3, x4, x5, x6))
#%timeit vec_func(x1, x2, x3, x4, x5, x6)

In [5]:
from sympy import lambdify
import numba
def make_gradient_from_graph(graph, wrt):
    wrt = tuple(wrt)
    varsig = ['float64({})'.format(','.join(['float64'] * len(wrt)))]
    grads = np.empty((len(wrt)), dtype=object)
    namespace = {}
    for i in range(len(wrt)):
        grads[i] = graph.diff(wrt[i])
        for j in range(i, len(wrt)):
            namespace['hess_{0}{1}'.format(i, j)] = numba.vectorize(lambdify(tuple(wrt), grads[i].diff(wrt[j]), dummify=True,
                                                              modules=[{'where': where}, 'numpy'], printer=NumPyPrinter))
        namespace['grad_{0}'.format(i)] = numba.vectorize(lambdify(tuple(wrt), grads[i], dummify=True,
                                                    modules=[{'where': where}, 'numpy'], printer=NumPyPrinter))

    grad_args = ','.join(['_x{0}'.format(i) for i in range(len(wrt))])
    grad_passed_args = ','.join(['_x{0}[0]'.format(i) for i in range(len(wrt))])
    grad_code = """def grad_func({0}, lengthfix, result):
    for i in range({1}):
        result[i] = grad({2})""".format(grad_args, len(wrt), grad_passed_args)
    grad_code = compile(grad_code, '<string>', 'exec')
    namespace = {'grad': namespace['grad_0']}
    exec grad_code in namespace
    grad_func = numba.guvectorize([','.join(['float64[:]'] * (len(wrt)+2))],
                                   ','.join(['()'] * len(wrt)) + ',(n)->(n)', nopython=True)(namespace['grad_func'])
    def hess_func(*args):
        result = np.empty(args[-1].shape + (len(args),len(args)))
        for i in range(len(args)):
            for j in range(i, len(wrt)):
                result[..., i, j] = result[..., j, i] = hess[i][j](*args)
        return result
    return grad_func, hess_func

In [6]:
%time grad_func, hess_func = make_gradient_from_graph(mod.ast, mod.variables)

CPU times: user 42.9 s, sys: 171 ms, total: 43.1 s
Wall time: 42.7 s


In [7]:
%time grad_func(x1, x2, x3, x4, x5, x6, np.empty(6), np.empty(x6.shape + (6,)))

CPU times: user 234 ms, sys: 1 ms, total: 235 ms
Wall time: 234 ms


array([[[[-23.32121246, -23.32121246, -23.32121246, -23.32121246,
          -23.32121246, -23.32121246],
         [-37.58299679, -37.58299679, -37.58299679, -37.58299679,
          -37.58299679, -37.58299679],
         [-45.67362128, -45.67362128, -45.67362128, -45.67362128,
          -45.67362128, -45.67362128],
         ..., 
         [-21.61568969, -21.61568969, -21.61568969, -21.61568969,
          -21.61568969, -21.61568969],
         [-15.63686798, -15.63686798, -15.63686798, -15.63686798,
          -15.63686798, -15.63686798],
         [-22.80037478, -22.80037478, -22.80037478, -22.80037478,
          -22.80037478, -22.80037478]],

        [[-34.26517587, -34.26517587, -34.26517587, -34.26517587,
          -34.26517587, -34.26517587],
         [ -2.99331956,  -2.99331956,  -2.99331956,  -2.99331956,
           -2.99331956,  -2.99331956],
         [-44.02807943, -44.02807943, -44.02807943, -44.02807943,
          -44.02807943, -44.02807943],
         ..., 
         [-27.61903919,

In [8]:
result = np.empty(x6.shape + (6,6))
%time hess_func(x1, x2, x3, x4, x5, x6)

NameError: global name 'hess' is not defined

In [ ]:
#%timeit pure_func(x1, x2, x3, x4, x5, x6)

In [ ]:
print(numba.typeof(grads))